In [27]:
%matplotlib inline

import zipline
from zipline.api import order_target_percent, symbol, set_commission, \
    set_slippage, schedule_function, date_rules, time_rules

from zipline.finance.commission import PerTrade, PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage

from datetime import datetime
import pytz
import pandas as pd
import numpy as np 

# To generate random numbers
from random import random, seed, randrange

"""
Settings
"""
number_of_runs = 2
random_portfolio_size = False
number_of_stocks = 50  # portfolio size, if not random    
sizing_method = 'equal' # equal or random

enable_commission = False
commission_pct = 0.001
enable_slippage = False 
slippage_volume_limit = 0.25
slippage_impact = 0.05

def initialize(context):
    # Fetch and store index membership
    context.index_members = pd.read_csv('../data/index_members/sp500.csv', index_col=0, parse_dates=[0])
    
    # Set commission and slippage.
    if enable_commission:
        comm_model = PerDollar(cost=commission_pct)
    else:
        comm_model = PerDollar(cost=0.0)
    set_commission(comm_model)
    
    if enable_slippage:
        slippage_model=VolumeShareSlippage(volume_limit=slippage_volume_limit, price_impact=slippage_impact)
        
    else:
        slippage_model=FixedSlippage(spread=0.0)  
    set_slippage(slippage_model)    
        
        
    schedule_function(
        func=rebalance,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open()
    )

def rebalance(context, data):
    today = zipline.api.get_datetime()
    
    # Check eligible stocks
    todays_universe = [
        symbol(ticker) for ticker in 
        context.index_members.loc[context.index_members.index < today].iloc[-1,0].split(',')
    ]
    
    # Make a list of stocks to buy
    buys = []
    
    # To modify global variable, and not make new one
    global number_of_stocks 
    
    # If random stockss selected
    if random_portfolio_size:
        # Buy between 5 and 200 stocks.
        number_of_stocks = randrange(5, 200)
    
    # Select stocks
    for i in np.arange(1, number_of_stocks +1):
        num = randrange(0, len(todays_universe) -1)
        buys.append(todays_universe.pop(num))
    
    # Sell positions no longer wanted.
    for security in context.portfolio.positions:
        if (security not in buys):
            order_target_percent(security, 0.0)
            
    #Make an empty DataFrame to hold target position sizes
    buy_size = pd.DataFrame(index=buys)
    
    # Get random sizes, if enabled.
    if sizing_method == 'random':
        buy_size['rand'] = [randrange(1,100) for x in buy_size.iterrows()]        
        buy_size['target_weight'] = buy_size['rand'] / buy_size['rand'].sum()
    elif sizing_method == 'equal':
        buy_size['target_weight'] = 1.0 / number_of_stocks
    
    # Send buy orders
    for security in buys:  
        order_target_percent(security, buy_size.loc[security, 'target_weight'])
        
start = datetime(1996, 1, 1, tzinfo=pytz.UTC)
end = datetime(2018, 12, 31, tzinfo=pytz.UTC)

# Empty DataFrame to hold the results
df = pd.DataFrame()

# Run the backtests
for i in np.arange(1, number_of_runs + 1):
    print('Processing run ' + str(i))
    
    result = zipline.run_algorithm(
        start=start, end=end, 
        initialize=initialize, 
        capital_base=100000,  
        data_frequency = 'daily', 
        bundle='ac_equities_db' )
    
    df[i] = result['portfolio_value']

print('All Done. Ready to analyze.')

Processing run 1


C:\ProgramData\Anaconda3_new\envs\zip35\lib\site-packages\empyrical\stats.py:790: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\ProgramData\Anaconda3_new\envs\zip35\lib\site-packages\empyrical\stats.py:704: RuntimeWarning: invalid value encountered in true_divide
  out=out,


Processing run 2
All Done. Ready to analyze.


In [26]:
# Save backtests to disk
df.to_csv("Random backtests.csv")


,1,2
1996-01-02 21:00:00+00:00,100000.000,100000.000
1996-01-03 21:00:00+00:00,100000.000,100000.000
1996-01-04 21:00:00+00:00,99093.711,98668.122
1996-01-05 21:00:00+00:00,98918.117,98572.710
1996-01-08 21:00:00+00:00,99034.949,98821.249
